# **이미지 모델링 실습**


## **1. 환경준비**

In [ ]:
# 라이브러리 로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

# 디바이스 준비
device = 'cuda' if torch.cuda.is_available() else 'cpu'

* 딥러닝을 위한 데이터로더 만들기

In [ ]:
def make_DataSet(x_train, x_val, y_train, y_val, batch_size = 32) :
    # 텐서로 변환
    x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)

    # TensorDataset 생성: 텐서 데이터 세트로 합치기
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

    # DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
    return train_loader, x_val_tensor, y_val_tensor

* 학습을 위한 함수

In [ ]:
def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset) # 전체 데이터 세트의 크기
    num_batches = len(dataloader) # 배치 크기
    tr_loss = 0

    model.train() # 학습 모드로 설정
    for x, y in dataloader: # 배치 단위로 로딩
        x, y = x.to(device), y.to(device) # 디바이스 지정

        # Feed Forward(오차 순전파)
        pred = model(x)
        loss = loss_fn(pred, y)
        tr_loss += loss

        # Backpropagation(오차 역전파)
        loss.backward() # 역전파를 통해 각 파라미터에 대한 오차의 기울기 계산
        optimizer.step() # 옵티마이저가 모델의 파라미터를 업데이트
        optimizer.zero_grad() # 옵티마이저의 기울기값 초기화.

    tr_loss /= num_batches # 모든 배치의 오차 평균
    return tr_loss.item()

* 검증을 위한 함수

In [ ]:
def evaluate(x_val_tensor, y_val_tensor, model, loss_fn, device):
    model.eval() # 모델을 평가 모드로 설정

    with torch.no_grad(): # 평가 과정에서 기울기를 계산하지 않도록 설정
        x, y = x_val_tensor.to(device), y_val_tensor.to(device)
        pred = model(x)
        eval_loss = loss_fn(pred, y).item() # 예측값 pred와 목푯값 y 사이의 오차 계산

    return eval_loss, pred

* 학습곡선

In [ ]:
def dl_learning_curve(tr_loss_list, val_loss_list):
    epochs = list(range(1, len(tr_loss_list)+1)) # 에포크 수 계산
    plt.plot(epochs, tr_loss_list, label='train_err', marker = '.') # 학습 오차 그래프
    plt.plot(epochs, val_loss_list, label='val_err', marker = '.') # 검증 오차 그래프
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

## **2. CNN 모델링 빠르게 구현하기**

### (1) 데이터 전처리

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

# 데이터 다운로드
train_set = datasets.MNIST(root="data", train=True, download=True, transform=ToTensor())
test_set = datasets.MNIST(root="data", train=False, download=True, transform=ToTensor())

# 학습 데이터의 데이터 로더 구성
batch_size = 64
train_loader = DataLoader(train_set, batch_size = batch_size)

# 검증, 테스트 데이터 준비
## 데이터 분할
x_val, x_test = test_set.data[:5000], test_set.data[5000:]
y_val, y_test = test_set.targets[:5000], test_set.targets[5000:]

## 스케일링
x_val = x_val / 255
x_test = x_test/ 255

## 이미지 처리를 위한 차원 맞추기
x_val = x_val.view(5000, 1, 28, 28)
x_test = x_test.view(5000, 1, 28, 28)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.55MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 130kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.19MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.87MB/s]


### (2) 모델링

* 모델 설계

In [ ]:
n_class = 10
model = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Flatten(),
            nn.Linear(32*14*14, 64),
            nn.ReLU(),
            nn.Linear(64, n_class)
            ).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

* 학습 및 검증 평가

In [ ]:
# 학습
epochs = 10
tr_loss_list, val_loss_list = [], []
for t in range(epochs):
    tr_loss = train(train_loader, model, loss_fn, optimizer, device)
    val_loss,_ = evaluate(x_val, y_val, model, loss_fn, device)
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)
    print(f"Epoch {t+1}, train loss : {tr_loss:.4f}, val loss : {val_loss:.4f}")

# 예측
_, pred = evaluate(x_test, y_test, model, loss_fn, device)
pred = nn.functional.softmax(pred, dim=1)
pred = np.argmax(pred.cpu().numpy(), axis = 1)

# 평가
print(classification_report(y_test.numpy(), pred, digits = 4))

Epoch 1, train loss : 0.2623, val loss : 0.1328
Epoch 2, train loss : 0.0836, val loss : 0.0928
Epoch 3, train loss : 0.0578, val loss : 0.0801
Epoch 4, train loss : 0.0435, val loss : 0.0767
Epoch 5, train loss : 0.0338, val loss : 0.0782
Epoch 6, train loss : 0.0260, val loss : 0.0807
Epoch 7, train loss : 0.0198, val loss : 0.0911
Epoch 8, train loss : 0.0153, val loss : 0.1024
Epoch 9, train loss : 0.0125, val loss : 0.1130
Epoch 10, train loss : 0.0100, val loss : 0.1133
              precision    recall  f1-score   support

           0     0.9923    0.9942    0.9933       520
           1     0.9877    0.9982    0.9929       564
           2     0.9783    0.9900    0.9842       502
           3     0.9921    0.9863    0.9892       510
           4     0.9979    0.9979    0.9979       482
           5     0.9863    0.9931    0.9897       436
           6     0.9919    0.9899    0.9909       496
           7     0.9922    0.9806    0.9864       516
           8     0.9817    0.993

## **3. MNIST 데이터로 CNN 모델 만들기 2(다중 합성곱층)**

* 모델 구조 설계

In [ ]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=32,
                                    kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64,
                                    kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

* 학습

In [ ]:
model2 = CNN_model().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model2.parameters(), lr=0.001)

In [ ]:
# 학습
epochs = 10
tr_loss_list, val_loss_list = [], []
for t in range(epochs):
    tr_loss = train(train_loader, model2, loss_fn, optimizer, device)
    val_loss,_ = evaluate(x_val, y_val, model2, loss_fn, device)
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)
    print(f"Epoch {t+1}, train loss : {tr_loss:.4f}, val loss : {val_loss:.4f}")

# 예측
_, pred = evaluate(x_test, y_test, model2, loss_fn, device)
pred = nn.functional.softmax(pred, dim=1)
pred = np.argmax(pred.cpu().numpy(), axis = 1)

# 평가
print(classification_report(y_test.numpy(), pred, digits = 4))

Epoch 1, train loss : 0.1787, val loss : 0.1006
Epoch 2, train loss : 0.0514, val loss : 0.0623
Epoch 3, train loss : 0.0330, val loss : 0.0523
Epoch 4, train loss : 0.0233, val loss : 0.0542
Epoch 5, train loss : 0.0192, val loss : 0.0570
Epoch 6, train loss : 0.0142, val loss : 0.0498
Epoch 7, train loss : 0.0111, val loss : 0.0711
Epoch 8, train loss : 0.0105, val loss : 0.0766
Epoch 9, train loss : 0.0083, val loss : 0.0735
Epoch 10, train loss : 0.0077, val loss : 0.0661
              precision    recall  f1-score   support

           0     0.9942    0.9962    0.9952       520
           1     1.0000    0.9929    0.9964       564
           2     0.9901    0.9940    0.9920       502
           3     0.9980    0.9941    0.9961       510
           4     1.0000    0.9959    0.9979       482
           5     0.9977    0.9931    0.9954       436
           6     0.9960    0.9980    0.9970       496
           7     0.9922    0.9903    0.9913       516
           8     0.9877    0.995

## **4. 층별 출력 크기 계산하기**

In [ ]:
class CNN_model2(nn.Module):
    def __init__(self):
        super(CNN_model2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
    def forward(self, x):
        print(f"입력 크기: {x.shape}")
        x = self.relu(self.conv1(x))
        print(f"Conv1 출력 크기: {x.shape}")
        x = self.pool1(x)
        print(f"MaxPool1 출력 크기: {x.shape}")
        x = self.relu(self.conv2(x))
        print(f"Conv2 출력 크기: {x.shape}")
        x = self.pool2(x)
        print(f"MaxPool2 출력 크기: {x.shape}")
        return x

In [ ]:
model2 = CNN_model2()
input = x_test[0:10]
output = model2(input)

입력 크기: torch.Size([10, 1, 28, 28])
Conv1 출력 크기: torch.Size([10, 32, 28, 28])
MaxPool1 출력 크기: torch.Size([10, 32, 14, 14])
Conv2 출력 크기: torch.Size([10, 64, 14, 14])
MaxPool2 출력 크기: torch.Size([10, 64, 7, 7])


## **5. MNIST 모델에 대한 수동 입력 테스트**

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
img = cv2.imread('02.png', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(255-img, (28, 28))
print(img.shape)
cv2_imshow(img)

In [ ]:
new_data = img.reshape(1,1,28,28)
new_data = new_data / 255
new_data = torch.tensor(new_data, dtype=torch.float32)

In [ ]:
model.eval() # 모델을 평가 모드로 전환
with torch.no_grad(): # 그래디언트 계산 비활성화(메모리 사용을 줄이고 속도 향상)
    new_data = new_data.to(device) # 입력 데이터를 모델과 동일한 디바이스로 이동
    pred = model(new_data) # 모델에 입력 데이터를 넣어 예측 수행
    pred = nn.functional.softmax(pred, dim=1) # 범주별 확률로 변환(소프트맥스 적용)
    pred = np.argmax(pred.cpu().numpy(), axis = 1) # 가장 확률 높은 범주의 인덱스 추출

print(pred)